# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [1]:
import pandas as pd
import numpy as np
import re

In [17]:
recipes = pd.read_csv("recipes_sample.csv", parse_dates = ["submitted"]).fillna(0.0)
random_rec = recipes[['id', 'minutes']].sample(n = 5)

output = []
for _, row in random_rec.iterrows():
    output.append(f"| {row['id']:^15} | {row['minutes']:^10} |")
    
print(f"| {'id':^15} | {'minutes':^10} |")
print(f"{'|-':-<31}|")
for line in output:
    print(line)

|       id        |  minutes   |
|------------------------------|
|     385172      |   40410    |
|      44845      |    190     |
|      21019      |     80     |
|      82065      |     26     |
|     161853      |     20     |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [22]:
from bs4 import BeautifulSoup


def show_info(name: str, steps: list, minutes: int, author_id: int):
    info_str = f'"{name.title()}"\n\n'
    for i, step in enumerate(steps, start=1):
        info_str += f"{i}. {step.capitalize()}\n"
    info_str += "----------\n"
    info_str += f"Автор: {author_id}\n"
    info_str += f"Среднее время приготовления: {minutes} минут\n"
    return info_str


id_170895 = recipes.loc[recipes['id'] == 170895]

with open('steps_sample.xml') as f:
    data = BeautifulSoup(f, 'xml')

steps = []
for recipe in data.recipes.find_all('recipe'):
    if recipe.find("id").next == '170895':
        steps = [step.next for step in recipe.steps.find_all('step')]
        break
        
info_str = show_info(
            name = id_170895['name'].values[0],
            steps = steps,
            minutes = id_170895['minutes'].values[0],
            author_id = id_170895['contributor_id'].values[0]
        )
print(info_str)

"Leeks And Parsnips  Sauteed Or Creamed"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an addi

In [7]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [23]:
with open('steps_sample.xml') as f:
    data = BeautifulSoup(f, 'xml')

pattern = r'\d+\shours|\d+\shour|\d+\sminutes|\d+\sminute'
steps = []

for recipe in data.recipes.find_all('recipe'):
    if recipe.find("id").next == '25082':
        steps = [step.next for step in recipe.steps.find_all('step')]

search = re.findall(pattern, '\n'.join(steps))
print(search)

['20 minutes', '10 minutes', '2 hours', '10 minutes', '20 minutes', '30 minutes']


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [24]:
pattern = r"^this[\w\s]*,\s?but"
search = recipes["description"].str.contains(pattern, regex=True).fillna(False)

print("Количество рецептов, содержащих шаблон:", search.sum())
print("Примеры рецептов, содержащих шаблон:\n")
for i, desc in recipes[search]["description"].head(3).items():
    print(f"Рецепт {i}:\n{desc}\n")

Количество рецептов, содержащих шаблон: 134
Примеры рецептов, содержащих шаблон:

Рецепт 76:
this is a great meal eaten the same day ,but even better the next day , if you can wait! add your favourite spices, but try it first as it is and i think that you will enjoy the 'vegetable' taste. good for freezing.

Рецепт 183:
this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers.

Рецепт 337:
this is kind of similar to some of the other versions out there, but it is the best and easiest i have found



5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [25]:
steps = []
for recipe in data.recipes.find_all('recipe'):
    if recipe.find("id").next == '72367':
        steps = [step.next for step in recipe.steps.find_all('step')]
        break

after_changes = re.sub(r'(\d+)\s*/\s*(\d+)', r'\1/\2', '\n'.join(steps))
print(after_changes)

mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [26]:
import nltk

with open('steps_sample.xml') as f:
    data = BeautifulSoup(f, 'xml')
    
recipes_and_steps = {}
for recipe in data.recipes.find_all('recipe'):
    steps = [step.next for step in recipe.steps.find_all('step')]
    recipes_and_steps[f'{recipe.find("id").next}'] = steps
    
words = []
for recipe_steps in recipes_and_steps.values():
    for step in recipe_steps:
        words.extend([word.lower() for word in nltk.word_tokenize(step) if word.isalpha()])

print("Количество уникальных слов:", len(set(words)))

Количество уникальных слов: 14926


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [27]:
long_sentences = []
for description in recipes['description']:
    sentences = nltk.sent_tokenize(str(description))
    long_sentences.append(len(sentences))

recipes['long_sentences'] = long_sentences
longest_descriptions = recipes.nlargest(5, 'long_sentences')
for i, row in longest_descriptions.iterrows():
    print(row)
    print()

name                   my favorite buttercream icing for decorating
id                                                           334113
minutes                                                          30
contributor_id                                               681465
submitted                                       2008-10-30 00:00:00
n_steps                                                        12.0
description       this wonderful icing is used for icing cakes a...
n_ingredients                                                   0.0
long_sentences                                                   76
Name: 18408, dtype: object

name              alligator claws  avocado fritters  with chipot...
id                                                           287008
minutes                                                          45
contributor_id                                               765354
submitted                                       2008-02-19 00:00:00
n_steps             

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [28]:
from nltk.tokenize import word_tokenize

def word_info(sentence):
    tokens = word_tokenize(sentence)
    tagged = nltk.pos_tag(tokens)
    speech_part = ''
    words = ''
    for tag in tagged:
        speech_part += f'{tag[1]}'.center(len(tag[0]) + 1)
        words += tag[0] + ' '
    print(speech_part)
    print(words)

recipe_241106 = recipes[recipes['id'] == 241106]
name = recipe_241106.loc[recipe_241106.index[0], 'name']
word_info(name)

    JJ     NNS    IN    NNS     VBP    JJ   CC   JJ    NNS  
eggplant steaks with chickpeas feta cheese and black olives 
